In [1]:
!pip install tensorflow
!pip install opencv-python
!pip install numpy

# **Fine tuning MobileNetV2 model**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np

# Load the pre-trained model (e.g., MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification: On Pavement or Outside Pavement
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


# **Data Processing**

In [ ]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image


image_path = ''
image = load_and_preprocess_image(image_path)

# **Train the Model**

In [ ]:
# training loop // has customized according to dataset structure
# Need to have lists of paths to images and corresponding labels

image_paths_on_pavement = [...]  # List of paths to images on the pavement
image_paths_outside_pavement = [...]  # List of paths to images outside the pavement

labels_on_pavement = [1] * len(image_paths_on_pavement)  # 1 for On Pavement
labels_outside_pavement = [0] * len(image_paths_outside_pavement)  # 0 for Outside Pavement

# Combine data from both classes
all_image_paths = image_paths_on_pavement + image_paths_outside_pavement
all_labels = labels_on_pavement + labels_outside_pavement

# Shuffle the data
combined_data = list(zip(all_image_paths, all_labels))
np.random.shuffle(combined_data)
all_image_paths, all_labels = zip(*combined_data)

# Train the model
model.fit(
    [load_and_preprocess_image(path) for path in all_image_paths],
    np.array(all_labels),
    epochs=10,  # Adjust as needed
    batch_size=32  # Adjust as needed
)


# **Model Evaluating and saving**

In [ ]:
# evaluation loop
# utilizing the separate validation set

# Loading validation dataset paths and labels
validation_image_paths = [...]  # List of paths to validation images
validation_labels = [...]  # List of validation labels

# Evaluate the model on the validation set
validation_loss, validation_accuracy = model.evaluate(
    [load_and_preprocess_image(path) for path in validation_image_paths],
    np.array(validation_labels)
)

print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

# Save the model
model.save('pedestrian_detection_model.h5')
